# Mirror Loop Analysis Demo

This notebook reproduces the canonical Mirror Loop curves from a cached dataset only. **No provider calls are made.**

- Input: `data/mirror_loop_results_all.csv`
- Outputs: `figures/fig_mirrorloop_curve.png`, `figures/fig_novelty_curve.png`
- Manuscript and prompts excluded during review

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
DATA = Path("data") / "mirror_loop_results_all.csv"
FIGS = Path("figures")
FIGS.mkdir(exist_ok=True, parents=True)

if not DATA.exists():
    print("⚠️  CSV not found. Using synthetic demo data.")
    import numpy as np
    demo_iters = np.arange(8)
    df = pd.DataFrame({
        "iteration": demo_iters,
        "edit_change": np.exp(-demo_iters / 3) + np.random.normal(0, 0.05, len(demo_iters)),
        "ngram_novelty": np.exp(-demo_iters / 2.5) + np.random.normal(0, 0.05, len(demo_iters))
    })
else:
    df = pd.read_csv(DATA)
    print(f"✓ Loaded {len(df)} rows from {DATA.name}")

In [ ]:
print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
# Aggregate across providers for the canonical pooled curve
pooled = df.groupby('iteration', as_index=False).agg(
    delta_I=('edit_change', 'mean'),
    ngram_novelty=('ngram_novelty', 'mean')
)
print(f"Pooled to {len(pooled)} iterations")
pooled

## ΔI Curve: Informational Decay and Grounding Rebound

In [ ]:
ax = pooled.plot(x='iteration', y='delta_I', marker='o', legend=False, figsize=(8,5))
ax.axvline(3, linestyle='--', color='gray', alpha=0.7, label='Minimal grounding (iter 3)')
ax.set_title("Mirror Loop: Informational Decay and Grounding Rebound (ΔI)")
ax.set_xlabel("Iteration")
ax.set_ylabel("ΔI (normalized edit distance)")
ax.legend()
fig_path = FIGS / "fig_mirrorloop_curve.png"
ax.figure.savefig(fig_path, bbox_inches='tight', dpi=150)
plt.show()
print(f"Saved: {fig_path}")

## Surface Novelty Decline

In [ ]:
ax2 = pooled.plot(x='iteration', y='ngram_novelty', marker='o', legend=False, color='coral', figsize=(8,5))
ax2.set_title("Surface Novelty Decline Across Iterations")
ax2.set_xlabel("Iteration")
ax2.set_ylabel("3-gram Novelty Ratio")
fig2_path = FIGS / "fig_novelty_curve.png"
ax2.figure.savefig(fig2_path, bbox_inches='tight', dpi=150)
plt.show()
print(f"Saved: {fig2_path}")

## Summary Statistics

In [ ]:
early = pooled.loc[pooled['iteration'].isin([1,2]), 'delta_I'].mean()
late  = pooled.loc[pooled['iteration'].isin([6,7]), 'delta_I'].mean()
reduction = ((late - early)/early)*-100

print(f"Mean ΔI early (1–2): {early:.3f}")
print(f"Mean ΔI late  (6–7): {late:.3f}")
print(f"Reduction: {reduction:.1f}%")
print(f"\nInterpretation: Mirror Loop demonstrates {reduction:.1f}% reduction in informational change,")
print("indicating recursive convergence toward a stable attractoR with minimal external grounding.")